In [34]:
import numpy as np
import pandas as pd

# Nygaard TDM

In [35]:
U_0 = 10
z_0 = 0.0001
D = 100
z_H = 100
C_T = 0.8
s_x = 10
s_y = 10
s_d = s_x*D
s_c = s_y*D
kappa = 0.4
rho = 1.225

In [36]:
def calculate_wind_speed(x_1, x_2, x_3, z_H, D, C_T, s_d, s_c):

    c_t = (np.pi * C_T * D**2) / (8 * s_d * s_c)

    nu_star = 28 * np.sqrt(c_t)
    
    beta =  nu_star / (1 + nu_star)
    
    z_0WF = z_H * (1 + D / (2 * z_H))**beta * np.exp(-kappa * (c_t + (kappa / np.log((z_H / z_0) * (1 - D / (2 * z_H))**beta))**2)**-0.5)

    x = np.linspace(x_1, x_3-1, int(x_3-x_1))

    h_1 = np.full_like(x, np.nan, dtype=float)
    h_2 = np.full_like(x, np.nan, dtype=float)

    alpha_def = 0.8

    #ratio = 1 + 13450 / x_3**1.28
    #alpha_rec = 0.8 * ratio
    #print('alpha_rec:', alpha_rec)
    alpha_rec = 0.8
    a = 1 #0.859

    h_1[x >= x_1] = z_H + a * ((x[x >= x_1] - x_1) / z_0WF)**alpha_def * z_0WF
    h_2[x >= x_2] =   0 + a * ((x[x >= x_2] - x_2) / z_0WF)**alpha_rec * z_0WF

    U = np.full_like(x, np.nan, dtype=float)
    U[0] = U_0

    idx1 = h_1 >= z_H
    U[idx1] = U_0 * (np.log(h_1[idx1] / z_0) * np.log(z_H / z_0WF)) / (np.log(h_1[idx1] / z_0WF) * np.log(z_H / z_0))

    idx2 = (x >= x_2) & (h_2 >= z_H)
    U[idx2] = U_0 * (np.log(h_1[idx2] / z_0) * np.log(h_2[idx2] / z_0WF)) / (np.log(h_1[idx2] / z_0WF) * np.log(h_2[idx2] / z_0))

    for i in range(1, len(U)):
        if U[i] >= U_0 * 0.9999:
            U[i] = U_0 * 0.9999
        elif U[i] < 0:
            U[i] = 0

    return x, U, c_t, idx2

# Alex's firkanter :)

In [37]:
d = 800 * D

l_f = int(10000)
w_f = int(10000 + (2*D/2))
h_f = D

x_1 = -l_f
x_2 = 0
x_3 = d

x, u_tdm, c_t, idx2 = calculate_wind_speed(x_1, x_2, x_3, z_H, D, C_T, s_d, s_c)

u_s_2 = (U_0 - u_tdm[idx2][0]) / U_0

u_s_3 = (U_0 - u_tdm[-1]) / U_0

N_wt = int( (l_f+s_d)/s_d * (w_f+s_c-(2*D/2))/s_c )

T = 1/2 * C_T * np.pi*(D**2)/4 * U_0**2 * N_wt #* rho (fjernet fra begge sider)

A = w_f * h_f
A_2 = T / (U_0**2 * (1 - u_s_2)*u_s_2)
A2_A = A_2 / A

A_3 = A_2 * ((1 - u_s_2)*u_s_2) / ((1 - u_s_3)*u_s_3)

A3_A2 = A_3 / A_2

ar = w_f / h_f

h_f_0 = np.sqrt(A_3 / ar)
w_f_0 = np.sqrt(A_3 * ar)

w_f_d = np.sqrt(A3_A2) * w_f_0
h_f_d = np.sqrt(A3_A2) * h_f_0

kx = (w_f_d - w_f_0) / (2*d*D)
#kx = (-h_f_0/4 - w_f_0/4 + (np.sqrt(h_f_0**2 + 2*h_f_0*w_f_0 + w_f_0**2 - 4*A_0 + 4*A_d))/4) / d*D
k = int( kx * d )

print(f'---------- {d/D:.0f} D ---------- \n')

print(f'c_t:     {c_t:.5f}      [-]')
print(f'N_wt:    {N_wt:.0f}          [-]')
print(f'T:       {T:.0f}     [kg·m·s⁻²]\n')

print(f'u_s_2:   {u_s_2:.5f}      [-]')
print(f'u_s_3:   {u_s_3:.5f}      [-]\n')

print(f'A:       {A:.0f}      [m²]')
print(f'A_2:     {A_2:.0f}      [m²]')
print(f'A2_A:    {A2_A:.1f}          [-]\n')

print(f'A_3:     {A_3:.0f}     [m²]')
print(f'A3_A2:   {A3_A2:.5f}     [-]\n')

print(f'kx:      {kx:.05f}      [-]')
print(f'k:       {k:.0f}          [m]')

---------- 800 D ---------- 

c_t:     0.00314      [-]
N_wt:    121          [-]
T:       38013271     [kg·m·s⁻²]

u_s_2:   0.16836      [-]
u_s_3:   0.00456      [-]

A:       1010000      [m²]
A_2:     2714999      [m²]
A2_A:    2.7          [-]

A_3:     83673062     [m²]
A3_A2:   30.81881     [-]

kx:      0.02615      [-]
k:       2092          [m]


# Save to CSV

In [38]:
y_meas = np.linspace(0 - k, w_f + k, w_f+(2*k)+1)
U = np.full_like(y_meas, u_tdm[-1])

In [39]:
data = pd.DataFrame({
    "y_wake": y_meas / D,
    "U_norm": U / U_0,
    "kx": kx,
})
data.to_csv(f"../ArbitraryLayouts/ValidationData/{int(d / D)}D.csv", index=False)